In [ ]:
import numpy as np
from numpy import array as arr
from numpy import zeros as zarr

import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.runSettings import load_settings
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr


import ROOT



def df_newcol(dframe, column, value, loc=-1):
    if column in dframe.columns and type(dframe.loc[0, column]) == type(value):
        print("WARNING: column", column, " already exists and has the same type")
        return
    if loc == -1:
        loc = len(dframe.columns)
    elif column in dframe.columns:
        loc = len(dframe.columns)
    dframe.insert(loc, column, pd.Series([value.copy()]*dframe.size, dtype=object))
    return

In [ ]:
MTPs = arr([5,6])

settingsDict = {
    "runID":190,
    "type":"TB",
}

frame = df(MTPs, columns=["MTP"])
df_newcol(frame, "b", settingsDict)
df_newcol(frame, "c", arr([1,2,3]), -1, )

index = frame.query("MTP == 5").index[0]
print(index)

frame.at[1, "c"] = [10,20,30]
frame

In [ ]:
runID = 190
MTPs = arr([5,6]) # MinimumTrackPlanes
thresholds = arr([180,200,210,220,230])
SECs = arr([0.0,0.1,0.2,0.3,0.4,0.5,0.6]) # SensorEdgeCut
runSettinsDict = load_settings(runID)

frame = df(MTPs, columns=["MTP"])
temp1 = df(thresholds, columns=['thr'])
frame = frame.merge(temp1, how='cross')
temp2 = df(SECs, columns=['SEC'])
frame = frame.merge(temp2, how='cross')
df_newcol(frame, "settings", runSettinsDict) # Column - runID
frame["runID"] = runID

FilePath = "/home/jona/DESY/analysis_TB/output/"
# frame.loc[3,"settings"]["filepath"] = FilePath

df_newcol(frame, "eff", [.0,.0,.0])
df_newcol(frame, "fakeRate", [.0,.0])

frame.query("MTP=={:n} & thr=={:n} & SEC=={:.1f}".format(5, 180, 0.0))

for MTP in MTPs:
    for thr in thresholds:
        for SEC in SECs:
            frameIndex = frame.query("MTP=={:n} & thr=={:n} & SEC=={:.1f}".format(MTP, thr, SEC)).index[0]
            FileName = FilePath+str(runID) + "_thr{:n}".format(thr) + "_ScanSensorEdge{:.1f}".format(SEC) + "_TrkMinPlanes{:n}".format(MTP) + ".root"
            frame.loc[3,"settings"]["filepath"] = FileName
            
            rFile = ROOT.TFile(FileName)
            if not rFile:
                print("File not found")
                rFile.Close()
                continue
            
            # Get the efficiency
            rDir = rFile.Get("AnalysisEfficiency/DSO9254A_0")
            if not rDir:
                print("Dir not found")
            else:
                TEff = rDir.Get("eTotalEfficiency")    
                if not TEff:
                    print("Efficiency not found")
                else:
                    frame.at[frameIndex, "eff"] = arr([TEff.GetEfficiency(1), TEff.GetEfficiencyErrorLow(1), TEff.GetEfficiencyErrorUp(1)])
                    
            # Get the fake rate
            rDir = rFile.Get("AnalysisEfficiency/DSO9254A_0/fake_rate")
            if not rDir:
                print("fake_rate: Dir not found")
            else:
                hFakePixelPerEvt = rDir.Get("hFakePixelPerEvent")
                if not hFakePixelPerEvt:
                    print("fake_rate: hFakePixelPerEvent not found")
                else:
                    frame.at[frameIndex, "fakeRate"] = arr([hFakePixelPerEvt.GetMean(), hFakePixelPerEvt.GetRMS()])
            
            rFile.Close()

In [ ]:
MTPs = arr([5,6])
thresholds = arr([180,220])

fig, ax = Hist.create_fig(len(MTPs), len(thresholds), figsize=(5,5), sharex=True, sharey=True)
ax2 = np.empty_like(ax, dtype=object)

for i_MTP, MTP in enumerate(MTPs):
    for i_thr, thr in enumerate(thresholds):
        runData = frame.query("MTP=={:n} & thr=={:n}".format(MTP, thr))
        runSettings = runData["settings"][runData.index[0]]
        axIndex = i_thr*len(thresholds) + i_MTP


        x = arr(runData["SEC"].to_list())
        y = arr(runData["eff"].to_list())
        yerr = arr([y[:,1], y[:,2]])
        y = y[:,0]
        
        ax[axIndex].errorbar(x, y, yerr=yerr, fmt='.-', color="black", lw=0.8, ms=2, zorder=200)
        # ax[axIndex].set_title("Min track planes: {:n} – thr: {:n} e".format(MTP, thr), fontdict={"fontsize":10})
        # ax[axIndex].text("Min track planes: {:n} – thr: {:n} e".format(MTP, thr), fontdict={"fontsize":10})
        text = "Min track planes: {:n} – thr: {:n} e".format(MTP, thr)
        ax[axIndex].text(0.5, 0.945, text, ha='center', va='center', transform=ax[axIndex].transAxes, fontdict={"size":9, "weight":"bold"})
        ax[axIndex].axhline(0.99, color="black", lw=0.5, ls="--", alpha=0.5)
        # ax.set_xlim(30,300)
        # ax[axIndex].set_ylim(0.7,1.1)
        
        ax2[axIndex] = ax[axIndex].twinx()
        color = 'darkred'
        ax2[axIndex].spines['right'].set_color(color)
        ax2[axIndex].tick_params(axis='y', labelcolor=color, direction="in", color=color)
        
        y = arr(runData["fakeRate"].to_list())*1000
        y_err = y[:,1]
        y = y[:,0]
        ax2[axIndex].errorbar(x, y, yerr=yerr, fmt='.-', color=color, lw=.8, ms=2, zorder=100)
        ax2[axIndex].set_ylim(0, 28)
        
Hist.finalize(runSettings, fig, ax, title="Sensor edge cut".format(thr), xlabel=r"Sensor edge cut / pixel pitch", ylabel="Efficiency", param_narrow=True, param_fontsize=8, grid=False, tight_layout=False, param_dynamic_placement=False)

ax[0].set_ylim(0.84, 1.02)
for i in range(2):
    ax2[2*i].set_yticklabels([])
    ax2[2*i+1].set_ylabel(r'Fake rate / $10^{-3}$ pix per evt', color=color, loc="top")
fig.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0.02, wspace=0.02)
fig.savefig("temp.pdf", pad_inches=0., bbox_inches="tight")
fig.savefig("temp.png", pad_inches=0., bbox_inches="tight", dpi=300)

In [ ]:
MTPs = arr([5])
SECs = arr([0.5])
thresholds = arr([180,200,210,220,230])

for MTP in MTPs:
    for SEC in SECs:
        
        # threshold scan
        
        fig, ax = Hist.create_fig(1,1, figsize=(5,4))
        
        x = thresholds
        y = arr(frame.query("MTP=={:n} & SEC=={:.1f}".format(MTP, SEC))["eff"].to_list())
        yerr = arr([y[:,1], y[:,2]])
        y = y[:,0]
        
        ax.errorbar(x, y, yerr=yerr, fmt='.-', color="black", lw=0.8, ms=2)